In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from networkx.algorithms import community as cm

from operator import itemgetter

from networkx import edge_betweenness_centrality as betweenness

### Remoção de arestas

O método da remoção de arestas consiste em ir removendo arestas da rede, de forma que as componentes resultantes formem comunidades. 

Um dos algoritmo mais utilizado para essa tarefa é o algoritmo de **Girvan-Newman** que remove as arestas que possuam maior centralidade de intermediação. 

Como critério de parada para o algoritmo pode se definir um valor máximo para a intermediação de uma aresta na rede resultante após cada passo.

O algoritmo continua o processo até que todos os nós fiquem isolados.

Graficamente, isto pode ser apresentado com um dendrograma.

A função **girvan_newman()** do pacote **networkx** implementa o métodp de **Girvan-Newman**. 

Por padrão, a função **girvan_newman** usa a centralidade de intermediação como métrica, mas é possível definir outra métrica para ser utilizada no método.

**nx.community()**: pacote do **networkx** com funções para calcular e medir a estrutura de comunidades de uma rede.

**nx.community.girvan_newman()**: função do **nx.community** para encontrar comunidades em um grafo usando o método **Girvan–Newman**

### Exemplo
Usando o grafo do karate club.

In [ ]:
#G = nx.barbell_graph(5,1)
G = nx.karate_club_graph()

fig, ax = plt.subplots(1,1,figsize=(15,10))

nx.draw(G,with_labels=True)

In [ ]:
comm = nx.community.girvan_newman(G)

top_level_comm = next(comm)

next_level_comm = next(comm)

print(sorted(map(sorted, top_level_comm)))
print(sorted(map(sorted, next_level_comm)))

In [ ]:
comm = nx.community.girvan_newman(G)

print(comm)

for it in comm:
    print(it)

In [ ]:
spec = nx.modularity_spectrum(G)
print(spec)


B = nx.modularity_matrix(G)
print(B)

### Exemplo:
Usando o grafo de caminho

In [ ]:
G = nx.path_graph(10)

fig, ax = plt.subplots(1,1,figsize=(15,10))

nx.draw(G,with_labels=True)

In [ ]:
comp = nx.community.girvan_newman(G)

#tuple(sorted(c) for c in next(comp))

tuple(sorted(c) for c in comp)

In [ ]:
# escolha de uma aresta a ser removida baseado no peso

edges = G.edges()

nx.set_edge_attributes(G, {(u, v): v for u, v in edges}, "weight")

def heaviest(G):
    u, v, w = max(G.edges(data="weight"), key=itemgetter(2))
    return (u, v)

comp = nx.community.girvan_newman(G, most_valuable_edge=heaviest)

#tuple(sorted(c) for c in next(comp))

tuple(sorted(c) for c in comp)

In [ ]:
# escolha da aresta a ser removida com maior centralidade de intermediação

def most_central_edge(G):
    centrality = betweenness(G, weight="weight")
    return max(centrality, key=centrality.get)

comp = nx.community.girvan_newman(G, most_valuable_edge=most_central_edge)

#tuple(sorted(c) for c in next(comp))

tuple(sorted(c) for c in comp)

In [ ]:
# Limita o número de comunidades a um número k
import itertools

k = 4 # numero de comunidades

comp = nx.community.girvan_newman(G)
limited = itertools.takewhile(lambda c: len(c) <= k, comp)

for communities in limited:
    print(tuple(sorted(c) for c in communities))


### Exemplo

In [ ]:
atri = pd.read_csv('../../data/harrypotter/hpattributes.txt', sep='\t')
ares = pd.read_csv('../../data/harrypotter/hpbook2.txt', sep=' ', header=None)
nome = pd.read_csv('../../data/harrypotter/hpnames.txt', sep='\t')

In [ ]:
# criando a rede

gpotter = nx.DiGraph()

n = atri.shape[0]

for k in range(n):
    gpotter.add_node(k,
                     nome = nome['name'][k],
                     ano = atri['schoolyear'][k],
                     gen = atri['gender'][k],
                     casa = atri['house'][k])

for k in range(n):
    for m in range(n):
        if ares.values[k][m] == 1:
            gpotter.add_edge(k,m)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,10))

nx.draw(gpotter,with_labels=True)

In [ ]:
# matriz de adjacencia esparsa
A = nx.to_scipy_sparse_array(gpotter)#.toarray()

# eliminando os nos isolados
isolados = []
for k in range(n):
    if np.sum(A[[k],:]) == 0:
        isolados.append(k)
        gpotter.remove_node(k)

print(len(isolados), 'nos isolados.')

print(f"#nodes de G = {nx.number_of_nodes(gpotter)}")
print(f"#edges de G = {nx.number_of_edges(gpotter)}")

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,10))

nx.draw(gpotter,with_labels=True)

In [ ]:
# matriz de adjacencia esparsa
#A = nx.adjacency_matrix(gpotter)
A = nx.to_scipy_sparse_array(gpotter)#.toarray()

# matriz de adjacencia densa
A = np.array(A.todense(),dtype=float)

No código a seguir a função **girvan_newman()** é aplicada a rede (tratada) de personagens do livro 2 da saga de Harry Potter.

In [ ]:
com = cm.girvan_newman(gpotter)
com = list(com)

In [ ]:
for it in gpotter.nodes():
    print(it, ':', gpotter.nodes[it]['nome'])

In [ ]:
for it in com:
    print(it)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,10))

plt.plot([-3.90380859375,-3.90380859375],[0,-20],'b',linewidth=5)
plt.plot([0,-47.8076171875,40,40],[-20,-20,-20,-20],'b', linewidth=5)
plt.plot([40,40],[-20,-400],'b', linewidth=5)
plt.plot([-47.8076171875,-47.8076171875],[-20,-40],'b', linewidth=5)
plt.plot([-75.615234375,-20],[-40,-40],'b', linewidth=5)
plt.plot([-20,-20],[-40,-400],'b', linewidth=5)
plt.plot([-75.615234375,-75.615234375],[-40,-60],'b', linewidth=5)
plt.plot([-111.23046875,-40],[-60,-60],'b', linewidth=5)
plt.plot([-40,-40],[-60,-400],'b', linewidth=5)
plt.plot([-111.23046875,-111.23046875],[-60,-80],'b',linewidth=5)
plt.plot([-152.4609375,-70],[-80,-80],'b', linewidth=5)
# plt.plot([-80,-80],[-80,-100],'b', linewidth=5)
plt.plot([-152.4609375,-152.4609375],[-80,-100],'b', linewidth=5)
plt.plot([-179.921875,-125],[-100,-100],'b', linewidth=5)
# plt.plot([-80,-80],[-80,-100],'b', linewidth=5)
plt.plot([-179.921875,-179.921875],[-100,-120],'b', linewidth=5)
plt.plot([-125,-125],[-100,-220],'b', linewidth=5)
plt.plot([-199.84375,-160],[-120,-120],'b', linewidth=5)
plt.plot([-160,-160],[-120,-400],'b', linewidth=5)
plt.plot([-199.84375,-199.84375],[-120,-140],'b', linewidth=5)
plt.plot([-219.6875,-180],[-140,-140],'b', linewidth=5)
plt.plot([-180,-180],[-140,-400],'b', linewidth=5)
plt.plot([-219.6875,-219.6875],[-140,-160],'b', linewidth=5)
plt.plot([-239.375,-200],[-160,-160],'b', linewidth=5)
plt.plot([-200,-200],[-160,-400],'b', linewidth=5)
plt.plot([-239.375,-239.375],[-160,-180],'b', linewidth=5)
plt.plot([-258.75,-220],[-180,-180],'b', linewidth=5)
plt.plot([-220,-220],[-180,-400],'b', linewidth=5)
# plt.plot([-400,-400],[-180,-200],'b', linewidth=5)
# plt.plot([-120,-120],[-100,-220],'b', linewidth=5)
plt.plot([-140,-120],[-220,-220],'b', linewidth=5)
plt.plot([-140,-140],[-220,-400],'b', linewidth=5)
plt.plot([10,10],[-20,-240],'b', linewidth=5)
plt.plot([00,20],[-240,-240],'b', linewidth=5)
plt.plot([0,0],[-240,-400],'b', linewidth=5)
plt.plot([20,20],[-240,-400],'b', linewidth=5)
plt.plot([-258.75,-258.75],[-180,-260],'b', linewidth=5)
plt.plot([-277.5,-240],[-260,-260],'b', linewidth=5)   
plt.plot([-240,-240],[-260,-400],'b', linewidth=5)

plt.plot([-277.5,-277.5],[-260,-280],'b', linewidth=5)
plt.plot([-295,-260],[-280,-280],'b', linewidth=5)
plt.plot([-260,-260],[-280,-400],'b', linewidth=5)

plt.plot([-70,-70],[-80,-300],'b', linewidth=5)
plt.plot([-80,-60],[-300,-300],'b', linewidth=5)
plt.plot([-80,-80],[-300,-400],'b', linewidth=5)
plt.plot([-60,-60],[-300,-400],'b', linewidth=5)

plt.plot([-295,-295],[-280,-320],'b', linewidth=5)
plt.plot([-315,-280],[-320,-320],'b', linewidth=5)
plt.plot([-280,-280],[-320,-400],'b', linewidth=5)
plt.plot([-315,-315],[-320,-340],'b', linewidth=5)

plt.plot([-340,-320],[-360,-360],'b', linewidth=5)
plt.plot([-340,-340],[-360,-400],'b', linewidth=5)
plt.plot([-320,-320],[-360,-400],'b', linewidth=5)

    
plt.plot([-330,-330],[-340,-360],'b', linewidth=5)
plt.plot([-330,-300],[-340,-340],'b', linewidth=5)
plt.plot([-300,-300],[-340,-400],'b', linewidth=5)


plt.plot([-140,-110],[-220,-220],'b', linewidth=5)
plt.plot([-110,-110],[-220,-380],'b', linewidth=5)
plt.plot([-120,-100],[-380,-380],'b', linewidth=5)
plt.plot([-120,-120],[-380,-400],'b', linewidth=5)
plt.plot([-100,-100],[-380,-400],'b', linewidth=5)

plt.box(False)
plt.xticks([])
plt.yticks([])
plt.ylim(-580,0)
plt.text(-340,-580,gpotter.nodes[28]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)
plt.text(-320,-580,gpotter.nodes[44]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.text(-300,-580,gpotter.nodes[25]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.text(-280,-580,gpotter.nodes[24]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.text(-260,-580,gpotter.nodes[19]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.text(-240,-580,gpotter.nodes[18]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.text(-220,-580,gpotter.nodes[ 2]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.text(-200,-580,gpotter.nodes[ 1]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.text(-180,-580,gpotter.nodes[55]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.text(-160,-580,gpotter.nodes[32]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.text(-140,-580,gpotter.nodes[10]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)
plt.text(-120,-580,gpotter.nodes[43]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)
plt.text(-100,-580,gpotter.nodes[57]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.text(- 80,-580,gpotter.nodes[20]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)
plt.text(- 60,-580,gpotter.nodes[51]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.text(- 40,-580,gpotter.nodes[27]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.text(- 20,-580,gpotter.nodes[23]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.text(-  0,-580,gpotter.nodes[13]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)
plt.text(+ 20,-580,gpotter.nodes[38]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.text(+ 40,-580,gpotter.nodes[16]['nome'],rotation='vertical',horizontalalignment='center',fontsize=14)

plt.show()